In [1]:
from datasets import load_dataset

dataset = load_dataset(
    "yitingxie/rlhf-reward-datasets",
    split="train"
)

print("Raw dataset size:", len(dataset))

C:\Users\ADMIN\anaconda3\envs\torch-gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\ADMIN\anaconda3\envs\torch-gpu\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ADMIN\.cache\huggingface\hub\datasets--yitingxie--rlhf-reward-datasets. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In ord

Raw dataset size: 76256


In [2]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
tokenizer.pad_token = tokenizer.eos_token

C:\Users\ADMIN\anaconda3\envs\torch-gpu\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ADMIN\.cache\huggingface\hub\models--gpt2-medium. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [3]:
MAX_TOKENS = 512
MIN_TOKENS = 10

def is_valid(example):
    if not all(k in example for k in ["prompt", "chosen", "rejected"]):
        return False

    prompt_len = len(tokenizer.encode(example["prompt"]))
    chosen_len = len(tokenizer.encode(example["chosen"]))
    rejected_len = len(tokenizer.encode(example["rejected"]))

    return (
        MIN_TOKENS < prompt_len < MAX_TOKENS and
        MIN_TOKENS < chosen_len < MAX_TOKENS and
        MIN_TOKENS < rejected_len < MAX_TOKENS
    )

In [4]:
filtered_dataset = dataset.filter(is_valid)
print("Filtered dataset size:", len(filtered_dataset))

Token indices sequence length is longer than the specified maximum sequence length for this model (1459 > 1024). Running this sequence through the model will result in indexing errors
Filter: 100%|████████████████████████████████████████████████████████████| 76256/76256 [01:24<00:00, 906.32 examples/s]

Filtered dataset size: 65351


In [5]:
reward_data = []

for ex in filtered_dataset:
    reward_data.append({
        "text": f"Human: {ex['prompt']}\nAssistant: {ex['chosen']}",
        "label": 1.0
    })
    reward_data.append({
        "text": f"Human: {ex['prompt']}\nAssistant: {ex['rejected']}",
        "label": -1.0
    })

In [6]:
from datasets import Dataset

reward_dataset = Dataset.from_dict({
    "text": [x["text"] for x in reward_data],
    "labels": [x["label"] for x in reward_data],
})

print("Reward dataset size:", len(reward_dataset))

Reward dataset size: 130702


In [7]:
reward_dataset = reward_dataset.shuffle(seed=42)
split = reward_dataset.train_test_split(test_size=0.1)

train_dataset = split["train"]
eval_dataset = split["test"]
print(len(train_dataset), len(eval_dataset))

117631 13071


In [8]:
def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

train_dataset = train_dataset.map(tokenize, batched=True, remove_columns=["text"])
eval_dataset = eval_dataset.map(tokenize, batched=True, remove_columns=["text"])

Map: 100%|██████████████████████████████████████████████████████████████| 13071/13071 [00:11<00:00, 1119.61 examples/s]


In [9]:
print(train_dataset[0].keys())
print(train_dataset[0]["labels"])

dict_keys(['labels', 'input_ids', 'attention_mask'])
1.0


# Reward Model Training

In [10]:
from transformers import GPT2ForSequenceClassification

model = GPT2ForSequenceClassification.from_pretrained(
    "gpt2-medium",
    num_labels=1
)

model.config.pad_token_id = tokenizer.pad_token_id

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2-medium and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./trained_reward_model",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,  # effective batch size = 16
    eval_strategy="steps",
    eval_steps=2000,
    save_steps=2000,
    save_total_limit=1,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=500,
    logging_steps=1000,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    report_to="none"
)

In [21]:
from transformers import Trainer, DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_15588\2686706032.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
2000,0.817600,1.050479
4000,0.832600,1.073771
6000,0.922100,0.972218
8000,0.889600,1.046347
10000,0.815000,1.048687
12000,0.812300,1.052472
14000,0.806200,1.082592
16000,0.659500,1.159615
18000,0.656400,1.187878
20000,0.658700,1.175401


In [18]:
!nvidia-smi

Wed Jan 14 19:51:51 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 576.88                 Driver Version: 576.88         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5070      WDDM  |   00000000:01:00.0 Off |                  N/A |
|  0%   37C    P8             11W /  250W |   10984MiB /  12227MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
trainer.save_model("C:/Users/ADMIN/Pictures/gpt2 finetune/trained_reward_model")
tokenizer.save_pretrained("C:/Users/ADMIN/Pictures/gpt2 finetune/trained_reward_model")